In [11]:

import numpy as np
import cv2
import imutils
from PIL import Image
import pytesseract
from gtts import gTTS
import IPython.display as ipd
import matplotlib.pyplot as plt
from ultralytics import YOLO

import torch

test_image = 'thaipublica_DSC_2081_รถเมล์.jpg'
img = Image.open(test_image)
img = np.array(img)

model = YOLO('runs/detect/train/weights/best.pt')
results = model(test_image)
result = results[0]
box = result.boxes[0]

import pandas as pd
df = pd.DataFrame(box.xyxy, columns=['x_min', 'y_min', 'x_max', 'y_max'])

# xmin
x_min = int(df['x_min'][0])

# xmax
x_max = int(df['x_max'][0])

# ymin
y_min = int(df['y_min'][0])

# ymax
y_max = int(df['y_max'][0])

# use numpy slicing to crop the region of interest
roi = img[y_min:y_max,x_min:x_max] 

# define a helper function to show image
def show_pic(img):
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')
    
# define helper functions to preprocess the ROI
# get grayscale image
def get_grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

# thresholding
def thresholding(image):
    thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return thresh

gray = get_grayscale(roi)
thresh = thresholding(gray)

number_list = ['1','2','3','4','5','6','7','8','9','0','-']
thai_characters = ['ก', 'ป', 'อ']
english_characters = ['A', 'E']

config_psm = '--psm 13'
text = pytesseract.image_to_string(thresh, config=config_psm)
new_text = ''

if not new_text:
    config_psm_count = 13
    while not new_text and config_psm_count >= 1:  # You can adjust the upper limit for config_psm_count as needed
        config_psm = f'--psm {config_psm_count}'
        text = pytesseract.image_to_string(thresh, config=config_psm)
        for i in text:
            if i in number_list or i in thai_characters or i in english_characters:
                new_text += i
        config_psm_count -= 1
        
audio_text = "รถเมล์หมายเลข " + new_text
language = 'th'
audio_obj = gTTS(text=audio_text, lang=language, slow=False) 
audio_obj.save("/untitled folder/bus_number.mp4") 


image 1/1 /Users/macjaru/Desktop/untitled folder/thaipublica_DSC_2081_รถเมล์.jpg: 448x640 1 number, 62.3ms
Speed: 1.7ms preprocess, 62.3ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)


In [12]:
ipd.Audio('/Users/macjaru/Desktop/untitled folder/bus_number.mp4')